In [64]:
# inputs and imports

import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv(r'C:\Users\Ferna\Python_Codes_2\wing_spar_bending_moments.csv')

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   posicao  991 non-null    float64
 1   momento  991 non-null    float64
dtypes: float64(2)
memory usage: 15.6 KB


In [66]:
df.head()

,posicao,momento
0,0.990,177.236946
1,0.991,177.236946
2,0.992,177.236946
3,0.993,177.236946
4,0.994,177.236946


In [67]:
def circular_section (D_ext,thickness,bending_moment,yield_strength):
    """Calculations of the inertial moments for a circular section according to ROARKS and SARKINS
    inputs in mm, N.m and MPa --> output in mm^4"""

    d_int = D_ext-(thickness*2)                     # internal diameter [mm]
    I_max = (3.14159*((D_ext**4)-(d_int**4)))/32    # obtained inertial moment [mm^4]

    c = D_ext/2     # neutral line
    CS = 1.5        # safety factor, defined by standard
    I_min = bending_moment*1e3*c/(yield_strength/CS)    # required moment of inertia

    # print('needed and obtained')
    return (I_min, I_max)

In [68]:
'''o calculo do momento inercial obtido nao bate com ROARKS nem SARKINS por um fator de 1/2'''
print(circular_section(20,1,15.88,273))
print(circular_section(12,1,15.88,273))

needed and obtained
(872.5274725274726, 5401.964005)
needed and obtained
(523.5164835164835, 1054.003445)


In [69]:
'''como seria o calculo pro nosso processo com o calculo atualizado, o projeto se manteria o mesmo'''
circular_section(22,2,200,270)

needed and obtained


(12222.222222222223, 12692.023599999999)

In [133]:
def rectangular_section (heigth,base,base_thickness,side_thickness,bending_moment,yield_strength):
    '''Calculations of the inertial moments for a rectangular section according to SARKINS
    inputs in mm, N.mm and MPa --> output in mm^4'''

    h = heigth - 2*base_thickness   # internal heigth
    b = base - 2*side_thickness     # internal base

    Jmax_x = round(((base*heigth**3)-(b*h**3))/12,3) # moment of inertia (x axis)
    Jmax_y = round(((heigth*base**3)-(h*b**3))/12,3) # moment of inertia (y axis)

    cv = heigth/2   # vertical neutral line
    ch = base/2     # horizontal neutral line
    CS = 1.5        # safety factor, defined by standard
    Jneeded_x = round(bending_moment*1e3*ch/(yield_strength/CS),3)    # needed inertial moment (x axis)
    Jneeded_y = round(bending_moment*1e3*cv/(yield_strength/CS),3)    # needed inertial moment (y axis)

    # print('needed in x, obtained in x, needed in y, obtained in y')
    return (Jneeded_x,Jmax_x,Jneeded_y,Jmax_y,)

In [145]:
rectangular_section(29,10,1,1,177.3,273)

(4870.879, 7202.167, 14125.549, 1264.667)

In [71]:
# calculo da constante torcional para uma seção retangular
# slideshare.net/JanelsonBrito/aula-02-torcao (slide 52)

def J_ret (B,H,esp_base,esp_lat):
    '''J = twisting constant or polar moment of inertia of the section [in millimeter]'''

    b_j = B-esp_lat     # base da secao media
    h_j = H-esp_base    # largura da secao media

    return round((2*b_j**2*h_j**2*esp_lat*esp_base)/(b_j*esp_lat+h_j*esp_base),3)

In [72]:
# another way to calculate the torcional constant
# https://app.uff.br/riuff/bitstream/handle/1/16135/PROJETO%20FINAL%20JOAO%20PAULO%20COUTINHO%20CARDOSO%20-%20Joao%20Paulo%20Coutinho%20Cardoso.pdf;jsessionid=EA9B4D10021E437D28A1D65DE44B877B?sequence=1

def J_general (A_m,per,t):
    '''general solution for the torcional constant
    A_m = average area
    per = perimeter
    t = thickness
    '''
    return (4*A_m**2)/(per/t)

In [73]:
# calculating the angle of twist, according to the following literature :
# https://wp.ufpel.edu.br/alinepaliga/files/2014/08/Tor%C3%A7%C3%A3o.pdf (slide 25)

def twist_angle (T,l,G,A_m,per,t):
    '''
    calculating the angle of twist of a section (radians)
    T = twisting moment
    l = length of the member
    J = twisting constant or polar moment of inertia of the section
    G = modulus of rigidity of the materia
    A_m = average area
    per = perimeter
    t = thickness
    [all the inputs in millimeter]    
    '''
    return (T*l)*per/(4*A_m**2*G*t)

In [74]:
# testing the formula seen in Roarks and in slideshare.net/JanelsonBrito/aula-02-torcao (slide 56)
# appling in the exercise seen in https://wp.ufpel.edu.br/alinepaliga/files/2014/08/Tor%C3%A7%C3%A3o.pdf (slide 26)

def twist_angle2 (T,l,G,J):
    '''
    calculating the angle of twist of a section (radians)  
    T = twisting moment
    l = length of the member
    J = twisting constant or polar moment of inertia of the section
    G = modulus of rigidity of the materia
    '''
    return (T*l)/(G*J)

In [ ]:
# exercicio https://wp.ufpel.edu.br/alinepaliga/files/2014/08/Tor%C3%A7%C3%A3o.pdf slide (25)
# resultado perfeito, mas a unnidade da tensao de cisalhamento esta em KPa ao inves de GPa (na propria resolucao do slide o autor faz isso)

twist_angle(85e3,1.5e3,26e3,50**2,50*4,10)

0.003923076923076923

In [ ]:
# exercicio https://wp.ufpel.edu.br/alinepaliga/files/2014/08/Tor%C3%A7%C3%A3o.pdf slide (26)
# resultado um pouco diferente, usando o valor médio de espessura de 4.2 (o resultado mais proximo seria com 3.8 de espessura)

twist_angle(35e3,2e3,38e3,35*57,2*35+2*57,4.2)

0.005069173835966163

In [ ]:
# utilizando o segundo algoritmo no mesmo exercicio anterior
# https://wp.ufpel.edu.br/alinepaliga/files/2014/08/Tor%C3%A7%C3%A3o.pdf slide (26)
# resultados um pouco diferentes

twist_angle2(35e3,2e3,38e3,J_ret(60,40,3,5))

0.005723367942323973

In [ ]:
# testando a constante com valores do TCC (pag 33)
# https://app.uff.br/riuff/bitstream/handle/1/16135/PROJETO%20FINAL%20JOAO%20PAULO%20COUTINHO%20CARDOSO%20-%20Joao%20Paulo%20Coutinho%20Cardoso.pdf;jsessionid=EA9B4D10021E437D28A1D65DE44B877B?sequence=1
# resultados coerentes --> incoerente com o primeiro codigo (montado com o JG)

print(str.format('valor encontrado pela func propria para secao retangular: {}',(J_ret(25,25,1.2,1.2))))
print(str.format('valor encontrado pela funcao desenvolvida com o JG: {}',rectangular_section(25,25,1.2,1.2)))
print(str.format('valor encontrado pela funcao geral: {}',J_general(25*25,4*25,1.2))) #FALTOU SUBTRAIRRRRRR

valor encontrado pela func propria para secao retangular: 16177.526

needed in x, obtained in x, needed in y, obtained in y
valor encontrado pela funcao desenvolvida com o JG: (187500.0, 10812.435, 187500.0, 10812.435)
valor encontrado pela funcao geral: 18749.999999999996


In [171]:
# torcional stiffness calculation by https://app.uff.br/riuff/bitstream/handle/1/16135/PROJETO%20FINAL%20JOAO%20PAULO%20COUTINHO%20CARDOSO%20-%20Joao%20Paulo%20Coutinho%20Cardoso.pdf;jsessionid=EA9B4D10021E437D28A1D65DE44B877B?sequence=1
def K (G,J,l):
    '''torcional stiffness calculation'''
    return round((G*J)/l,3)

In [77]:
# testing for the SAE paper
print(str.format('valor encontrado pela func propria para secao retangular: {}',(J_ret(20,20,6.3,6.3))))
print(str.format('valor encontrado pela funcao geral: {}',J_general(13.7*13.7,4*13.7,6.3)))
print(str.format('valor encontrado pela funcao desenvolvida com o JG: {}',rectangular_section(20,20,6.3,6.3,200,273)))

valor encontrado pela func propria para secao retangular: 16199.524
valor encontrado pela funcao geral: 16199.523899999997
needed in x, obtained in x, needed in y, obtained in y
valor encontrado pela funcao desenvolvida com o JG: (10.989, 13083.445, 10.989, 13083.445)


In [ ]:
# testing the variations of stiffness for differents J's
print(K(26e3,J_ret(50,50,10,10),1.5e3))
print(K(26e3,J_general(40*40,4*40,10),1.5e3))

11093333.333
11093333.333


In [192]:
# por que eu fiz isso??? eu nao preciso disso kk

def p0(wingspan,fuselage_width,first_profile_spacing):
    '''function to find the initial position of the bending moment
    inputs in meter --> index in df'''

    p0 = wingspan/2 + fuselage_width/2 + first_profile_spacing 
    return df.posicao.to_list().index(p0)

print(p0(1.98,0.17,0.004))
print(df.momento[p0(1.98,0.17,0.004)])

89
177.23694626866842


In [193]:
def square_optimization (lamination_thickness,minimal_thickness_addimited,tensile_strength_expected):
    '''optimization for a square wing spar profile
    inputs in mm, MPa--> list'''

    width_x_list=[] # list of the best horizontal width for each point after the fuselage
    torcional_stiffness_list=[]

    for moment in df['momento']:
        for x in range (minimal_thickness_addimited,48):
            if rectangular_section(x,x,lamination_thickness,lamination_thickness,moment,tensile_strength_expected)[0] < rectangular_section(x,x,lamination_thickness,lamination_thickness,moment,tensile_strength_expected)[1]:
                list.append(width_x_list, x)
                torcional_stiffness = K(tensile_strength_expected*1e6,J_ret(x,x,lamination_thickness,lamination_thickness),df.posicao[df.momento.to_list().index(moment)])
                list.append(torcional_stiffness_list,torcional_stiffness)
                break
    return (width_x_list, torcional_stiffness_list)

df['square_optimization'] = square_optimization(2,10,135)[0]
df['torcional_stiffness_for_square_profile'] = square_optimization(2,10,135)[1]
df

,posicao,momento,square_optimization,torcional_stiffness_for_square_profile
0,0.990,177.236946,31,6.651545e+12
1,0.991,177.236946,31,6.644834e+12
2,0.992,177.236946,31,6.651545e+12
3,0.993,177.236946,31,6.644834e+12
4,0.994,177.236946,31,6.644834e+12
...,...,...,...,...
986,1.976,0.046484,10,6.995951e+10
987,1.977,0.034863,10,6.992413e+10
988,1.978,0.023242,10,6.988878e+10
989,1.979,0.011621,10,6.985346e+10


18.287588294651865

In [197]:
# torcional stiffness in m^4
# considering 270 MPa

k0 = K(270e6,J_general(df.square_optimization.mean()*df.square_optimization.mean(),4*df.square_optimization.mean(),2)*1e-12,0.99)

def vd ():
    '''velocidade de divergincia'''

    k0
    p = 1.1994 # densidade do ar no local da competicao
    S = 0.765 # Area da asa
    e = 0.001 # Distancia entre os centros elastico e aerodinamico da asa (segundo Silva)
    Cla = 3.576

    return (k0/(p * S * e * Cla))**(1/2)

vd()

31.886108980162636

In [194]:
rectangular_section(20,20,6.3,6.3,200,273)[0]

10989.011

In [195]:
df.posicao[0]

0.99